In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentencepiece
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
# tested in transformers==4.18.0, pytorch==1.7.1
import torch
import transformers
torch.__version__, transformers.__version__

('2.5.0+cu121', '4.44.2')

In [5]:
torch.cuda.is_available()

False

## Load dataset

In [6]:
df = pd.read_csv('/content/drive/MyDrive/xian/sentences_with_sentiment.csv') ## use your own customized dataset
df = df[['text', 'sentiment']]
df.head()

,text,sentiment
0,Yes.,0
1,Our solutions address those challenges by rapi...,1
2,"On the film side, we are seeing an unprecedent...",1
3,"And with that, I'd like to turn the call over ...",0
4,Welcome to our fourth quarter and full year 20...,0


In [7]:
# Split into training+validation and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)

# Split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

In [8]:
df_train, df_test, = train_test_split(df, stratify=df['sentiment'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['sentiment'],test_size=0.1, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(420, 2) (52, 2) (47, 2)


## Load FinBERT model

In [9]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length' , max_length=128), batched=True)


dataset_train = dataset_train.rename_column("sentiment", "labels")
dataset_val = dataset_val.rename_column("sentiment", "labels")
dataset_test = dataset_test.rename_column("sentiment", "labels")

# Adjust labels for all datasets
dataset_train = dataset_train.map(lambda x: {'labels': x['labels'] + 1})
dataset_val = dataset_val.map(lambda x: {'labels': x['labels'] + 1})
dataset_test = dataset_test.map(lambda x: {'labels': x['labels'] + 1})


dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

## Define training options

In [15]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
    output_dir='temp/',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    save_safetensors=False  # Disable safe serialization
)


trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,            # evaluation dataset
        compute_metrics=compute_metrics
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.735760,0.659574


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.735760,0.659574
2,No log,0.627888,0.723404
3,No log,0.579227,0.723404
4,No log,0.541906,0.787234
5,No log,0.531922,0.808511


TrainOutput(global_step=70, training_loss=0.6138792310442244, metrics={'train_runtime': 2999.5791, 'train_samples_per_second': 0.7, 'train_steps_per_second': 0.023, 'total_flos': 138134544307200.0, 'train_loss': 0.6138792310442244, 'epoch': 5.0})

In [16]:
# Model Evaluation
model.eval()
trainer.predict(dataset_test).metrics

{'test_loss': 0.5121446847915649,
 'test_accuracy': 0.7884615384615384,
 'test_runtime': 43.2366,
 'test_samples_per_second': 1.203,
 'test_steps_per_second': 0.046}

In [17]:
# Save the model locally
trainer.save_model('/content/drive/MyDrive/xian/finbert-sentiment/')

## Push the model to the Hub

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [23]:
import os
import torch

# Define a function to save the model with contiguous tensors
def save_contiguous_model(model, save_directory):
    os.makedirs(save_directory, exist_ok=True)
    # Get the model's state dict and make each tensor contiguous
    state_dict = model.state_dict()
    contiguous_state_dict = {k: v.contiguous() for k, v in state_dict.items()}

    # Save using the modified state dict
    torch.save(contiguous_state_dict, os.path.join(save_directory, "pytorch_model.bin"))
    model.config.save_pretrained(save_directory)

# Make tensors contiguous and save
save_contiguous_model(model, "finbert-base-sentiment")
tokenizer.save_pretrained("finbert-base-sentiment")


('finbert-base-sentiment/tokenizer_config.json',
 'finbert-base-sentiment/special_tokens_map.json',
 'finbert-base-sentiment/vocab.txt',
 'finbert-base-sentiment/added_tokens.json')

In [26]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Reload the model and tokenizer from the saved directory
model = AutoModelForSequenceClassification.from_pretrained("finbert-base-sentiment")
tokenizer = AutoTokenizer.from_pretrained("finbert-base-sentiment")

# Push to the hub
model.push_to_hub("hzduuuu/finbert-base-sentiment")
tokenizer.push_to_hub("hzduuuu/finbert-base-sentiment")

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/hzduuuu/finbert-base-sentiment/commit/948567a3220171a5e97c7ae586056d6b35f14513', commit_message='Upload tokenizer', commit_description='', oid='948567a3220171a5e97c7ae586056d6b35f14513', pr_url=None, pr_revision=None, pr_num=None)